# **Multi-GPU Numerical Computing: Threading + RMM + Numba CUDA**

## **Prerequisites**

This tutorial assume proficiency in Python and the following libraries:

* Threading
* Numba
* NumPy

Demo System - Benchmarking was performed on a DGX Station A100 320GB

## **Why Multi-GPU Numba CUDA?**

[Numba CUDA Python](https://numba.readthedocs.io/en/stable/cuda/index.html) has been used to GPU accelerate code without leaving Python. This is extremely compelling for those performing rapid prototyping or maintain a desire to stay in Python. Most examples and applications of Numba are single GPU. This notebook demonstrates achieving higher Numba CUDA kernel performance on a compute intensive workload using multiple GPUs and the [RAPIDS Memory Manager (RMM)](https://github.com/rapidsai/rmm).

**When to consider this programming pattern:**
1. Kernel needs to run faster and is saturating a single GPU
2. Two independent kernels need to run simultaneously without competing for resources
3. Streaming data needs to be loaded into GPU and processed rapidly by an expensive kernel

**Note:**
1. Sending data between devices implies an I/O penalty and overhead -- performance improvements will be most pronounced if already saturating a single GPU (for smaller problems, overheads could dominate performance)
2. Use managed memory allocation strategy to share data between multiple GPUs
3. Data can also be shared between GPUs using use Numba's APIs page-locked ([pinned](https://numba.pydata.org/numba-doc/dev/cuda-reference/memory.html#numba.cuda.pinned_array) or [mapped](https://numba.pydata.org/numba-doc/dev/cuda-reference/memory.html#numba.cuda.mapped_array), but traversed the PCIe bus (not shown here)
4. Only on Numba CUDA context can be active at a time per CPU thread

*Inspired by this example:
https://github.com/ContinuumIO/numbapro-examples/tree/master/multigpu*

## **Problem Overview**

In this notebook, we leverage a proxy geospatial nearest neighbor problem to guide us through an evaluation of a tailor-made multi-GPU technique using Python threading + RMM + Numba CUDA. In this use case, we aim to resolve geospatial observations to their nearest reference points with an added complexity. Our complication adds dynamics to the problem allowing each reference point to move and the set of observations to change on a reoccurring basis. These complexities imply a need to recompute each nearest neighbor at each timestep -- emphasizing the need for high performance techiques. 

Because of its simplicity and arithmetic intensity, we focus our attention on the brute force nearest neighbor technique using the haversine great circle distance formula as our distance metric. This is a popular formula used to calculate the distance between two points on earth.

<center><a href="https://en.wikipedia.org/wiki/Haversine_formula"><img src="./media/haversine-graphic.png" alt="Haversine" style="width: 150;"></a></center></br>

The graphic below illustrates the dynamic nature of our problem. From left to write, we can observe the dynamics of the system at each timestep -- with colored regions representing nearest neighbor decision boundaries for each reference point and points representing observations.

<center><img src="./media/DynamicDecisionBoundaries.png" alt="Visualization" style="width: 1000;"/></center>

In this notebook, we will use all available GPUs (4xA100) on a problem scaled up by 2048x - 8.8T

**Spoiler Alert -- This GPU technique out performs the parallel CPU example by almost 600x.**

<center><img src="./media/AllScaledCpuGpuPerfTable.png" alt="PerfTable" style="width: 1000;"/></center>

 # **Multi-GPU Experiment**

In [ ]:
from numba import cuda
import cupy as cp
import numba as nb
import rmm
import cudf

from src.solvers import (block_min_reduce,
                         global_min_reduce)

from src.simulator import generate_geos
from src.utils import check_accuracy
import math
import threading
import queue
import numpy as np

Define constants for the size of our experiment and evaluation criteria.

In [ ]:
N_OBS, N_REF = 2**27, 2**16 # single processor experiment
N_OBS_VAL, N_REF_VAL = 500, 200 # check accuracy
print("Problem Size (N_OBS * N_REF): {:.2f}T".format(N_OBS * N_REF * 1e-12))

## **RAPIDS Memory Manager**

Use RMM to define a managed memory GPU allocation strategy, making it easy handle multi-GPU communications over NVLink/NVSwitch.

In [ ]:
# use managed memory for allocations
cuda.set_memory_manager(rmm.RMMNumbaManager)

rmm.mr.set_current_device_resource(
    rmm.mr.ManagedMemoryResource())

## **Define Thread Function**

The ```_get_nearest_multi``` thread function will perform a double Numba CUDA kernel launch pattern outlined in the single gpu/cpu notebook. Here we launch kernels to operate on partitions of data defined in a job configuration object ```q```.  Each thread has a unique CUDA context defined by GPU index position ```cid```.

In [ ]:
def _get_nearest_multi(q, cid):
        
    cuda.select_device(cid) # bind device to thread
        
    while(q.unfinished_tasks > 0):

        job = q.get()
                
        d_ref = cuda.to_device(
            job["d_m_ref"]
        )
        
        d_obs = cuda.to_device(
            job["d_m_obs"][job["start"]:job["end"]]
        )
        
        d_block_idx = cuda.device_array(
            (job["end"] - job["start"], 32), 
            dtype=np.uint32)
        
        d_block_dist = cuda.device_array(
            (job["end"] - job["start"], 32), 
            dtype=np.float32)        
                
        bpg = 32, 108
        tpb = 32, 16

        block_min_reduce[bpg, tpb](
            d_ref,
            d_obs,
            d_block_idx,
            d_block_dist           
        )
        
        bpg = (1, 108*20)
        tpb = (32, 16)

        global_min_reduce[bpg, tpb](
            d_block_dist,
            d_block_idx,
            job["d_m_out_dist"][job["start"]:job["end"]],
            job["d_m_out_idx"][job["start"]:job["end"]]
        )        

        cuda.synchronize()

        q.task_done()

## **Define Multi-GPU Function**

Based on available GPU configuration, build up a queue ```queues``` of jobs (kernel launches on partitions) to be scheduled in each CPU thread. Our thread function ```_get_nearest_multi``` grabs jobs from this queue and performs the work in its own CPU thread.

In [ ]:
def get_nearest(
    obs_points, ref_points,
    batch_size="auto",
    multigpu=True,
    n_gpus="auto"):
    
    out_idx = cuda.device_array(
        (obs_points.shape[0]),
        dtype=np.uint32)
    
    out_dist = cuda.device_array(
        (obs_points.shape[0]),
        dtype=np.float32)
    
    if not multigpu:
        
        d_block_idx = cuda.device_array(
            (out_idx.shape[0], 32), 
            dtype=np.uint32)
        
        d_block_dist = cuda.device_array(
            (out_idx.shape[0], 32), 
            dtype=np.float32)           
        
        bpg = 32, 108
        tpb = 32, 16
              
        block_min_reduce[bpg, tpb](
            ref_points, 
            obs_points, 
            d_block_idx,
            d_block_dist
        )   
        
        bpg = (1, 108*20)
        tpb = (32, 16)        

        global_min_reduce[bpg, tpb](
            d_block_dist, 
            d_block_idx, 
            out_dist, 
            out_idx
    )   
        
        cuda.synchronize()
        
        return out_idx, out_dist

    if n_gpus == "auto":
        n_gpus = len(cuda.list_devices())
        
    size = obs_points.shape[0]
        
    if batch_size == 'auto':
        batch_size = size/(n_gpus)
        
    batch_size = int(batch_size)

    n_jobs = int(size / min(batch_size, size))
        
    queues = [queue.Queue() for i in range(n_gpus)]
    
    qid = 0
    
    for j in range(n_jobs):
        
        if qid >= len(queues):
            qid = 0

        job = {}
        
        start = j * batch_size
        
        if j == (n_jobs - 1):
            end = size
        else:
            end = (j + 1) * batch_size
        
        job["start"] = start
        job["end"] = end
        job["d_m_ref"] = ref_points
        job["d_m_obs"] = obs_points
        job["d_m_out_idx"] = out_idx
        job["d_m_out_dist"] = out_dist        
        
        queues[qid].put(job)
        
        qid += 1
    
    workers = []
    
    for qid in range(len(queues)):
        
        w = threading.Thread(
            target=_get_nearest_multi, 
            args=[queues[qid], 
                  qid])
        
        w.start()
        workers.append(w)
        
    for w in workers:
        w.join()     
        
    return out_idx, out_dist

## **Generate Dataset**

Let's generate a scaled up synthetic dataset and validation dataset for our work today using an included utility function. These datasets represent the following:

* ```d_obs``` contains ```N_OBS``` geospatial obserations in radians on the GPU, used for our full scale benchmark
* ```d_ref``` contains ```N_REF``` geospatial reference points in radians on the GPU, used for our full scale benchmark
* ```d_obs_val``` contains ```N_OBS_VAL``` observations in radians on the GPU, used to validate accuracy
* ```d_ref_val``` contains ```N_REF_VAL``` geospatial reference points in radians on the GPU, used to validate accuracy

In [ ]:
d_ref = generate_geos(N_REF, random_state=1)
d_obs = generate_geos(N_OBS, random_state=2)

d_ref_val = generate_geos(N_REF_VAL, random_state=1)
d_obs_val = generate_geos(N_OBS_VAL, random_state=2)

## **Validate Accuracy**

Verify our multi-GPU method is producing the correct results.

In [ ]:
d_idx_val, d_dist_val = get_nearest(d_obs_val, d_ref_val, 
            batch_size='auto', 
            multigpu=True, n_gpus="auto")

print("Accuracy - Threading RMM Numba CUDA Multi-GPU:", 
      check_accuracy(
          d_obs_val, 
          d_ref_val,
          d_idx_val, 
          d_dist_val)
     )

## **Benchmark Performance**

Our kernel completes in 12.7s on the demo system, ~584x faster than the parallel CPU alternative!

In [ ]:
%%timeit
d_idx, d_dist = get_nearest(
    d_obs, d_ref, 
    batch_size='auto',
    multigpu=True, 
    n_gpus="auto"
)

# **Summarize Results**

In summary, we observe our multi-GPU technique solves our scaled up problem orders of magnitude faster than the parallel CPU alternative. This implementation required more developer effort than the ```dask_cudf``` implementation, but achieved slightly higher performance for this use case. We also benefit from more control over problems with n-dimensional arrays.

<img src="./media/MultiScaledCpuGpuPerfTable.png" alt="PerfTable" style="width: 150;"/>


<br>
<div align="left"><h2><b>Please Restart the Kernel<b></h2></div>

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)